# 一、逻辑回归

逻辑回归主要是处理二分类问题，假设我们的score function采用线性函数表示，即 $s = X \cdot W$,其中X为$N \times D$(N个样本，每个样本D维)，而W为$D \times 1$

那么我们所求得的 $s = \{s_1,s_2,...,s_N \}^T$, 其中 $s_j$表示第j个输入的score。

我们使用sigmoid函数对输入的score function 进行处理，那么所有的score都处于0到1之间
而我们只有两个分类，$C_1$,$C_2$

令   
$$p(C_1 \vert x) = \sigma (x\cdot W)$$
那么
$$p(C_2 \vert x) = 1- p(C_1 \vert x)$$




## 逻辑回归的损失函数及其对W的偏导数


对于一个数据集 $\{X_n,t_n\}$,其中 $t_n \in {0,1},n = 1,2...N$

其似然函数可以写作：

$$p(\mathbf{t} \vert \mathbf{w}) = \prod_{n=1}^{N}y_n^{t_n}\{1-y_n\}^{1-t_n}$$

其中  $\mathbf{t} = \{t_1,....t_N\}^T$ 且  $y_n = p(C_1 \vert x_n)$

我们将loss function定为似然函数的负对数，这会导出 cross-entropy loss function:

$$E(\mathbf{w}) = -lnp(\mathbf{t} \vert \mathbf{w}) = - \sum_{n=1}^{N} \{ t_n ln y_n + (1-t_n) ln (1-y_n)\}$$

我们使用链式法则对其求对W的偏导数。

我们定义 $y_n = \sigma (a_n)$,$a_n = W^Tx_n$

$$\frac{\partial E}{\partial y_n} = \frac{1-t_n}{1-y_n} - \frac{t_n}{y_n} = \frac{y_n -t_n}{y_n ( 1-y_n )}
$$

$$\frac{\partial y_n}{\partial a_n} =\frac{\partial \sigma (a_n)}{\partial a_n} = y_n(1-y_n)$$

且
$$\frac{\partial a_n}{\partial W} = x_n$$

那么 $$\nabla E(W) = \sum_{n=1}^{N}(y_n - t_n)x_n$$

我们可以看到  

> 损失函数对参数W的偏导是输入的线性组合，每个输入样本$x_n$对偏导的贡献系数为分类错误 $y_n -t_n$

> 我们将看到很多损失函数的偏导都有着上述形式


> 工程实现时，我们可以如下计算

$$\nabla E(W) =[x_1,x_2,...x_N]\begin{bmatrix}y_1-t_1 \\ y_2-t_2 \\ \vdots \\ y_n-t_n \end{bmatrix}= X^T(\mathbf{y}-\mathbf{t})$$